# Computational Methods in Astrophysics: Assignment 3
##### Prasad Rajesh Posture
Roll No.: MSC2303121013

# Importing Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation
from IPython import display
%matplotlib tk

#### Total Variation Diminishing Schemes
**Total Variation:** It is a measure of how much a function oscillates or varies within a given interval. It is given as $$TV(q) = \sum_{i=0}^{N} |q_{i} - q_{i-1}|$$
**TVD Scheme:**
A numerical scheme is said to be total variation diminishing (TVD) scheme if
$$ TV(q^{n+1}) \leq TV(q^n)$$

Such a scheme will not develop oscillations near a jump, because a jump is a monotonically
in/decreasing function and a total variation diminishing scheme will not increase the TV ,
and hence in this case keep the TV identically constant. This appears to be a desirable property
of the numerical scheme.

#### Slope Limiters
Sharp changes or discontinuities in a solution can introduce errors in numerical simulations because traditional numerical methods, like finite difference or finite volume schemes, struggle to accurately represent and resolve abrupt changes in the solution. When a sharp change occurs, the solution gradients become very steep, leading to large numerical errors and oscillations in the computed solution. These errors arise because the numerical method may not capture the rapid changes adequately, leading to inaccuracies in the computed values. **Slope limiters** help mitigate these issues by smoothing out the solution near sharp changes, thereby reducing the steepness of the gradients and suppressing oscillations. They restrict the steepness of the solution gradients to avoid unphysical behavior like overshoots and oscillations, especially in regions of sharp gradients or discontinuities.

#### Flux Limiters
Flux limiters are primarily used to control the propagation of information across grid cells in hyperbolic PDEs. They limit the magnitude of the numerical fluxes to prevent the formation of spurious oscillations and ensure stability and accuracy of the solution, particularly in regions of steep gradients or discontinuities. Flux limiters are essential for capturing shock waves and other discontinuities accurately, as they help maintain the monotonicity and positivity of the solution by controlling the fluxes at cell interfaces.

#### Difference between Slope Limiter and Flux Limiter
1) Slope limiters are used to control the steepness of solution gradients within individual grid cells. Flux limiters, on the other hand, are used to control the propagation of information across grid cells in hyperbolic.<br>
2) PDEs.Slope Limiters: Applied within individual grid cells to control the rate of change of the solution. Flux Limiters: Applied at cell interfaces to control the magnitude of fluxes between adjacent grid cells.<br>
3) Slope Limiters: Smoothen out the solution gradients within cells, preventing unphysical oscillations near sharp changes. Flux Limiters: Control the propagation of information between grid cells, ensuring stability and accuracy of the solution by limiting the magnitude of fluxes, especially in regions of steep gradients or discontinuities.

#### Defining all the required functions

In [49]:
# Defining Phi's: Flux Limiters
# Defining flux limiter functions
# works
def donner_cell(r):
    "Donner Cell"
    return 0.0
# works
def lax_wendroff(r):
    "Lax-Wendroff"
    return 1.0

# works
def beam_warming(r):
    "Beam-Warming"
    return r

# works
def fromm(r):
    "Fromm"
    return (1/2)*(1+r)

# works
def minmod(r):
    "Minmod"
    if r>0:
        if 1<=abs(r):
            return 1
        elif 1>abs(r):
            return r
        else:
            pass
    else:
        return 0

# works
def superbee(r):
    "Superbee"
    return max(0, min(1,2*r), min(2,r))

# No reference to verify
# It is some similiar to superbee, given that the superbee isnt working properly
# Same issue could be with this one
def mc(r):
    "MC"
    return max(0, min((1+r)/2, 2, 2*r))

# no reference 
# lets assume that it works
def van_leer(r):
    "van Leer"
    return (r+abs(r))/(1+abs(r))


In [50]:
# Defining a solver function

def pde_solver(method, nt=10):
    global method_, dt_, nt_
    method_ = method
    nx = 200
    x = np.linspace(-10, 10, nx)
    dx = x[1] - x[0]
    dt = 10**(-3) 
    dt_ = dt    
    nt_ = nt                                          
    t = np.arange(0,nt, dt)          
    c = 3.0

    pulse_height = 1.0
    pulse_width = 5.0
    q0 = np.where((np.abs(x) <= pulse_width), pulse_height, 0)
    
    q = q0.copy()
    sol = []
   
    for i in range(len(t)):
        qn = q.copy()
        sol.append(qn)
       
        for i in range(nx):
            theta = 1
        
            dm = ((qn[i] - qn[(i-1) % nx]))
            if dm != 0:
                rnm = (qn[(i - 1) % nx] - qn[(i - 2) % nx]) / dm
            else:
                rnm = 0.0
            phim = method(rnm)
            
            dp = ((qn[(i+1)%nx] - qn[i]))
            if dp !=0:
                rnp = (qn[i] - qn[(i - 1) % nx]) / dp
            else:
                rnp = 0.0
            phip = method(rnp)

            
            fnm = c/2 * ((1 + theta)*qn[(i - 1) % nx] + (1 - theta)*qn[i]) + abs(c)/2 * (1-abs(c*dt/dx))*(qn[i] - qn[(i - 1) % nx]) * phim
            fnp = c/2 * ((1 + theta)*qn[i] + (1 - theta)*qn[(i + 1) % nx]) + abs(c)/2 * (1-abs(c*dt/dx))*(qn[(i + 1) % nx] - qn[i]) * phip
            
            q[i] = qn[i] + (dt/dx) * (fnm-fnp)
    return sol

In [51]:
# Defininga a plotter function
def animator(sol):
    x = np.linspace(-10,10,len(sol[0]))
    fig = plt.figure()
    lines = plt.plot([])
    line = lines[0]
    plt.xlim(-10,10)
    plt.ylim(-0.05, max(sol[0])+0.1)
    plt.ylabel("Amplitude")
    plt.xlabel("Position(x)")
    plt.title(method_.__doc__)
    def animate(frame):
        y = sol[frame*int((nt_/dt_)/(40*nt_))]
        line.set_data((x,y))
    anim = FuncAnimation(fig, animate, frames=40*nt_, interval=25)
    return anim, plt.show()

#### Running the simulation for each method

In [52]:
animator(pde_solver(donner_cell))

(<matplotlib.animation.FuncAnimation at 0x179cf3e8440>, None)

In [53]:
animator(pde_solver(lax_wendroff))

(<matplotlib.animation.FuncAnimation at 0x179d52177a0>, None)

In [54]:
animator(pde_solver(beam_warming))

(<matplotlib.animation.FuncAnimation at 0x179d63cf050>, None)

In [55]:
animator(pde_solver(fromm))

(<matplotlib.animation.FuncAnimation at 0x179d76cc590>, None)

In [56]:
animator(pde_solver(minmod))

(<matplotlib.animation.FuncAnimation at 0x179c852a3c0>, None)

In [57]:
animator(pde_solver(superbee))

(<matplotlib.animation.FuncAnimation at 0x179d76c5850>, None)

In [58]:
animator(pde_solver(mc))

(<matplotlib.animation.FuncAnimation at 0x179cf543500>, None)

In [59]:
animator(pde_solver(van_leer))

(<matplotlib.animation.FuncAnimation at 0x179e0a8b080>, None)

# 2. 1D Isothermal Shock Tube